In [1]:
import os
import sys
import struct
from inspect import getmembers, isfunction
from typing import Iterable, Tuple, Any, Dict

import soundfile
import config as cfg

import librosa
import numpy as np
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
from Settings import *
import pandas as pd
from auditok import ADSFactory, AudioEnergyValidator, StreamTokenizer, player_for, dataset

Using TensorFlow backend.


In [2]:
def convertPCM(destPath, srcPath):
    #print(srcPath)
    outPath = destPath
    fout = open(outPath,'wb') #用二进制的写入模式
    #fout.write(struct.pack('4s','\x66\x6D\x74\x20'))
    #写入一个长度为4的串，这个串的二进制内容为 66 6D 74 20
    #Riff_flag,afd,fad,afdd, = struct.unpack('4c',fin.read(4))
    #读入四个字节，每一个都解析成一个字母
    #open(sys.argv[4],'wb').write(struct.pack('4s','fmt '))
    #将字符串解析成二进制后再写入
    #open(sys.argv[4],'wb').write('\x3C\x9C\x00\x00\x57')
    #直接写入二进制内容：3C 9C 00 00 57
    #fout.write(struct.pack('i',6000)) #写入6000的二进制形式
    #check whether inFile has head-Info
    inPath= srcPath
    fin = open(inPath,'rb')
    Riff_flag, = struct.unpack('4s',fin.read(4))
    if Riff_flag == 'RIFF':
        #print("%s have head" % inPath)
        fin.close()
        #sys.exit(0)
    else:
        #print("%s no head" % inPath)
        fin.close()
        #采样率
        sampleRate = int(16000)
        #bit位
        bits = int(16)
        fin = open(inPath,'rb')
        startPos = fin.tell()
        fin.seek(0,os.SEEK_END)
        endPos = fin.tell()
        sampleNum = int(endPos - startPos)
        #print(sampleNum)
        #headInfo = geneHeadInfo(sampleRate,bits,sampleNum)
        #fout.write(headInfo)
        fout.write('\x52\x49\x46\x46'.encode())
        fout.write(struct.pack('i',sampleNum + 36))
        #fout.write(fileLength)
        fout.write('\x57\x41\x56\x45\x66\x6D\x74\x20\x10\x00\x00\x00\x01\x00\x01\x00'.encode())
        fout.write(struct.pack('i',sampleRate))
        fout.write(struct.pack('i',int(sampleRate * bits / 8)))
        fout.write('\x02\x00'.encode())
        fout.write(struct.pack('H',bits))
        fout.write('\x64\x61\x74\x61'.encode())
        fout.write(struct.pack('i',sampleNum))
        fin.seek(os.SEEK_SET)
        fout.write(fin.read())
        fin.close()
        fout.close()
        # We set the `record` argument to True so that we can rewind the source
        asource = ADSFactory.ads(filename=destPath, record=True)
        validator = AudioEnergyValidator(sample_width=asource.get_sample_width(), energy_threshold=50)
        # Default analysis window is 10 ms (float(asource.get_block_size()) / asource.get_sampling_rate())
        # min_length=20 : minimum length of a valid audio activity is 20 * 10 == 200 ms
        # max_length=400 :  maximum length of a valid audio activity is 400 * 10 == 4000 ms == 4 seconds
        # max_continuous_silence=30 : maximum length of a tolerated  silence within a valid audio activity is 30 * 30 == 300 ms
        tokenizer = StreamTokenizer(validator=validator, min_length=50, max_length=400, max_continuous_silence=1)
        asource.open()
        tokens = tokenizer.tokenize(asource)
        # Play detected regions back
        #player = player_for(asource)
        #print("\n ** playing detected regions...\n")
        data = b''
        
        for i,t in enumerate(tokens):
            #print("Token [{0}] starts at {1} and ends at {2}".format(i+1, t[1], t[2]))
            data = data + b''.join(t[0])
        #player.play(data)

        sampleNum = len(data)
        if sampleNum>1000:
            #采样率
            sampleRate = int(16000)
            #bit位
            bits = int(16)
            fout = open(srcPath+".wav",'wb')
            fout.write('\x52\x49\x46\x46'.encode())
            fout.write(struct.pack('i',sampleNum + 36))
            #fout.write(fileLength)
            fout.write('\x57\x41\x56\x45\x66\x6D\x74\x20\x10\x00\x00\x00\x01\x00\x01\x00'.encode())
            fout.write(struct.pack('i',sampleRate))
            fout.write(struct.pack('i',int(sampleRate * bits / 8)))
            fout.write('\x02\x00'.encode())
            fout.write(struct.pack('H',bits))
            fout.write('\x64\x61\x74\x61'.encode())
            fout.write(struct.pack('i',sampleNum))
            fout.write(data)
            fout.close()
            #assert len(tokens) == 8
            asource.close()

In [3]:
def read_audio(path, target_fs=None):
    if path.upper().endswith(".PCM"):
        convertPCM("tt.wav", path)

In [4]:
def audios_to_features(files: Iterable[str]) -> np.ndarray:
    """
    Extract audio features from a collection of files.
    :param files: The files
    :return: An array of array of features
    """
    return np.asarray([read_audio(file) for file in files])

In [5]:
def f(path):
    if path.endswith(AUDIO_EXT_PCM) == False:
        return False
    return True

def list_files(dir_name: str, ext=AUDIO_EXT) -> np.ndarray:
    """
    List the files in a directory recursively for a given extension.
    :param dir_name: The directory to search
    :param ext: The extension of the files to search for
    :return: The array of filenames
    """
    return np.asarray(list(map(lambda path: path.replace("\\", PATH_SEPARATOR),
                               filter(lambda path: f(path),
                                      [os.path.join(dp, f) for dp, dn, fn in os.walk(dir_name) for f in fn]))))

In [19]:
audios_to_features(list_files("test"))

array([None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None, None, None, None, None, None, None,
       None, None, None, None, None], dtype=object)

In [18]:
audios_to_features(list_files("data/Train/Train/FAPB0"))

array([None, None, None, None, None, None], dtype=object)

In [6]:
audios_to_features(list_files("data/Train/Train"))

array([None, None, None, ..., None, None, None], dtype=object)

In [ ]:
def convertPCM(destPath, srcPath):
    #print(srcPath)
    outPath = destPath
    fout = open(outPath,'wb') #用二进制的写入模式
    #fout.write(struct.pack('4s','\x66\x6D\x74\x20'))
    #写入一个长度为4的串，这个串的二进制内容为 66 6D 74 20
    #Riff_flag,afd,fad,afdd, = struct.unpack('4c',fin.read(4))
    #读入四个字节，每一个都解析成一个字母
    #open(sys.argv[4],'wb').write(struct.pack('4s','fmt '))
    #将字符串解析成二进制后再写入
    #open(sys.argv[4],'wb').write('\x3C\x9C\x00\x00\x57')
    #直接写入二进制内容：3C 9C 00 00 57
    #fout.write(struct.pack('i',6000)) #写入6000的二进制形式
    #check whether inFile has head-Info
    inPath= srcPath
    fin = open(inPath,'rb')
    Riff_flag, = struct.unpack('4s',fin.read(4))
    if Riff_flag == 'RIFF':
        #print("%s have head" % inPath)
        fin.close()
        #sys.exit(0)
    else:
        #print("%s no head" % inPath)
        fin.close()
        #采样率
        sampleRate = int(16000)
        #bit位
        bits = int(16)
        fin = open(inPath,'rb')
        startPos = fin.tell()
        fin.seek(0,os.SEEK_END)
        endPos = fin.tell()
        sampleNum = int(endPos - startPos)
        #print(sampleNum)
        #headInfo = geneHeadInfo(sampleRate,bits,sampleNum)
        #fout.write(headInfo)
        fout.write('\x52\x49\x46\x46'.encode())
        fout.write(struct.pack('i',sampleNum + 36))
        #fout.write(fileLength)
        fout.write('\x57\x41\x56\x45\x66\x6D\x74\x20\x10\x00\x00\x00\x01\x00\x01\x00'.encode())
        fout.write(struct.pack('i',sampleRate))
        fout.write(struct.pack('i',int(sampleRate * bits / 8)))
        fout.write('\x02\x00'.encode())
        fout.write(struct.pack('H',bits))
        fout.write('\x64\x61\x74\x61'.encode())
        fout.write(struct.pack('i',sampleNum))
        fin.seek(os.SEEK_SET)
        fout.write(fin.read())
        fin.close()
        fout.close()

def read_audio(path, target_fs=None):
    if path.upper().endswith(".PCM"):
        convertPCM("tt.wav", path)
        # We set the `record` argument to True so that we can rewind the source
        asource = ADSFactory.ads(filename="tt.wav", record=True)
        validator = AudioEnergyValidator(sample_width=asource.get_sample_width(), energy_threshold=65)
        # Default analysis window is 10 ms (float(asource.get_block_size()) / asource.get_sampling_rate())
        # min_length=20 : minimum length of a valid audio activity is 20 * 10 == 200 ms
        # max_length=400 :  maximum length of a valid audio activity is 400 * 10 == 4000 ms == 4 seconds
        # max_continuous_silence=30 : maximum length of a tolerated  silence within a valid audio activity is 30 * 30 == 300 ms
        tokenizer = StreamTokenizer(validator=validator, min_length=50, max_length=400, max_continuous_silence=1)
        asource.open()
        tokens = tokenizer.tokenize(asource)
        # Play detected regions back
        #player = player_for(asource)
        #print("\n ** playing detected regions...\n")
        data = b''
        for i,t in enumerate(tokens):
            #print("Token [{0}] starts at {1} and ends at {2}".format(i+1, t[1], t[2]))
            data = data + b''.join(t[0])
        #player.play(data)
        #采样率
        sampleRate = int(16000)
        #bit位
        bits = int(16)
        sampleNum = len(data)
        fout = open(TEMP_WAV,'wb')
        fout.write('\x52\x49\x46\x46'.encode())
        fout.write(struct.pack('i',sampleNum + 36))
        #fout.write(fileLength)
        fout.write('\x57\x41\x56\x45\x66\x6D\x74\x20\x10\x00\x00\x00\x01\x00\x01\x00'.encode())
        fout.write(struct.pack('i',sampleRate))
        fout.write(struct.pack('i',int(sampleRate * bits / 8)))
        fout.write('\x02\x00'.encode())
        fout.write(struct.pack('H',bits))
        fout.write('\x64\x61\x74\x61'.encode())
        fout.write(struct.pack('i',sampleNum))
        fout.write(data)
        fout.close()
        #assert len(tokens) == 8
        asource.close()
        #player.stop()
        path = TEMP_WAV
    (audio, fs) = soundfile.read(path)
    if audio.ndim > 1:
        audio = np.mean(audio, axis=1)
    if target_fs is not None and fs != target_fs:
        audio = librosa.resample(audio, orig_sr=fs, target_sr=target_fs)
        fs = target_fs
    return audio, fs